In [1]:
from __future__ import print_function

import os
import sys
import warnings

import pystan
import matplotlib.pyplot as plt
import numpy as np

sys.path.append(os.environ['HOME']+'/home/proj/stats/MCMC')
# sys.path.append(os.environ['HOME']+'/home/proj/stats/MCMC/submmSED')

import data

%matplotlib inline

In [2]:
herus_file = "august6table.csv"
speed_of_light = 299792.458 ### micron GHz
d = data.readfluxes_DLC_2014(filename=herus_file)

IOError: [Errno 2] No such file or directory: 'august6table.csv'

In [3]:
def makeStanDict(dobj, N_comp=2):
    """ convert from an MCMC object to a dict for input to STAN"""
    return dict(
        N_comp = N_comp,
        N_band = dobj.n,
        nu_obs = dobj.freq_obs,
        flux = dobj.d,
        sigma = dobj.sig,
        z = dobj.z
    )

In [4]:
submm_data = """
    data {

        int<lower=1> N_comp; // number of greybody components (fixed model parameter)

        int<lower=1> N_band;   // number of photometric bands
        vector[N_band] nu_obs; // observed frequency
        vector[N_band] flux;   // observed flux
        vector[N_band] sigma;  // error
        real z;                // redshift
    }
    
    transformed data {
        vector[N_band] nu;     // rest frame frequency
        nu <- (1+z)*nu_obs;    // could do this outside of stan...
    }
    
"""

submm_functions = """
    functions {
        
        real greybody(real beta, real T, real nu) {
          // greybody, normalized to unit flux at nu=nu_0
            real h_over_k;
            real x;
            real nu_0;
            real x_0;
            
            nu_0 <- 1000;

            h_over_k <- 0.04799237;       //  K/Ghz
            x <- h_over_k * nu / T;
            x_0 <- h_over_k * nu_0 / T;
            return (pow(nu/nu_0, 3+beta) * expm1(x_0) / expm1(x));
        }
    }
"""

submm_model_greybody_1 = """
    // single-component greybody -- not used
    parameters {
        real amplitude<lower=0.0>;
        real T<lower=3, upper=100.0>; // Temperature in K
        real beta<lower=0, upper=3>;  // greybody factor
    }
    
    transformed parameters {
        real totalflux<lower=0>[N_band];
        for (band in 1:N_band)
            totalflux[band] <- amplitude * greybody(beta, T, nu[band])
    }
    
    model {
        // no explicit priors on parameters -- improper... 
        flux ~ normal(totalflux, sigma)
    }
    
"""

submm_model_greybody = """
    parameters {
    // nb. N_comp, N_band are data
//        vector[N_comp] log10amplitude;
        vector<lower=0>[N_comp] amplitude;
//          vector<lower=3, upper=100.0>[N_comp] T; // Temperature in K  
        positive_ordered[N_comp] T;
        vector<lower=0, upper=3>[N_comp] beta;  // greybody factor    
    }
    
// not used, since this is only needed if you want to keep track of these parameters.
//    transformed parameters {
//        real<lower=0> fluxes[N_band, N_comp];
//        real<lower=0> totalflux[N_band];
//        real<lower=0> amplitude[N_comp];
//        
//        for (comp in 1:N_comp)
//            amplitude[comp] <- pow(10.0, log10amplitude[comp]);
//        
//        for (band in 1:N_band) {
//            for (comp in 1:N_comp) {  // vectorize over this?
//                fluxes[band, comp] <- amplitude[comp] * greybody(beta[comp], T[comp], nu[band]);
//            }
//           totalflux[band] <- sum(fluxes[band]);
//        }
//    }
    
    model {
        real fluxes[N_band, N_comp];
        vector[N_band] totalflux;
//        vector[N_comp] amplitude;
        
//        for (comp in 1:N_comp)
//            amplitude[comp] <- pow(10.0, log10amplitude[comp]);
        
        for (band in 1:N_band) {
            for (comp in 1:N_comp) {  // vectorize over this?
                fluxes[band, comp] <- amplitude[comp] * greybody(beta[comp], T[comp], nu[band]);
            }
            totalflux[band] <- sum(fluxes[band]);
        }

        // no explicit priors on parameters -- improper... 
       flux ~ normal(totalflux, sigma);
    }
    
"""


submm_model_greybody_b2 = """

//// like submm_model_greybody, but with fixed beta=2 
    parameters {
    // nb. N_comp, N_band are data
        vector<lower=0>[N_comp] amplitude;
//          vector<lower=3, upper=100.0>[N_comp] T; // Temperature in K  
        positive_ordered[N_comp] T;
    }

    model {
        real fluxes[N_band, N_comp];
        vector[N_band] totalflux;

        for (band in 1:N_band) {
            for (comp in 1:N_comp) {
                fluxes[band, comp] <- amplitude[comp] * greybody(2.0, T[comp], nu[band]);
            }
            totalflux[band] <- sum(fluxes[band]);
        }

        // no explicit priors on parameters -- improper... 
       flux ~ normal(totalflux, sigma);
    }
    
"""

In [5]:
greybody_code = submm_functions + submm_data + submm_model_greybody
greybody_model = pystan.StanModel(model_code=greybody_code)
greybody_b2_code = submm_functions + submm_data + submm_model_greybody_b2
greybody_b2_model = pystan.StanModel(model_code=greybody_b2_code)

In [6]:
obj = 7
dobj = d[obj]
N_comp = 2
print(d[obj].name)

fit = greybody_model.sampling(data=makeStanDict(d[obj], N_comp=N_comp),
                  iter=2000, chains=4)


IRAS06206-6315


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/queues.py:392: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/queues.py:392: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/queues.py:392: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing

In [13]:
print(fit)
fit.summary()

Inference for Stan model: anon_model_62ed89537f2de8a0302771c2118de22b.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
amplitude[0]   0.41  3.8e-3   0.07   0.31   0.36   0.39   0.45   0.55  298.0   1.01
amplitude[1]   0.18  3.7e-3   0.06   0.04   0.14    0.2   0.23   0.28  297.0   1.01
T[0]          20.37    0.07   1.55  17.81  19.26  20.19  21.34  23.81  443.0   1.01
T[1]          54.62    0.31   7.58  40.61   49.3  54.26  59.48   69.8  585.0   1.01
beta[0]        2.68    0.01   0.22   2.24    2.5   2.71   2.87   2.99  375.0   1.01
beta[1]         1.5    0.03   0.47    0.9   1.17   1.38   1.72   2.72  324.0   1.01
lp__          -7.74    0.07   1.78 -12.12   -8.7  -7.41  -6.45  -5.27  570.0    1.0

Samples were drawn using NUTS(diag_e) at Wed Aug 12 13:12:17 2015.
For each parameter, n_eff is a crude measure of effective sample size,


OrderedDict([('c_summary',
              array([[[  4.04154197e-01,   6.54516129e-02,   4.06913181e-01,
                         6.31067831e-02],
                      [  4.16818261e-01,   6.53475582e-02,   4.06773340e-01,
                         6.54876482e-02],
                      [  3.12137328e-01,   3.56453724e-01,   3.86760325e-01,
                         4.43129921e-01],
                      [  5.50612010e-01,   3.04376941e-01,   3.58046181e-01,
                         3.94565948e-01],
                      [  4.52075079e-01,   5.39963291e-01,   3.17533222e-01,
                         3.67217537e-01],
                      [  4.02398462e-01,   4.60072560e-01,   5.56814553e-01,
                         3.11900830e-01],
                      [  3.56638325e-01,   3.91523511e-01,   4.52974757e-01,
                         5.45826286e-01]],
              
                     [[  1.85702349e-01,   6.50059837e-02,   1.83381872e-01,
                         6.23358316e-02],
     

In [7]:
fits = {}
niter = 10000
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    for dobj in d:
        print(dobj.name)
        fits[dobj.name] = []
        for n_comp in (1,2):
            fit_beta = greybody_model.sampling(data=makeStanDict(dobj, N_comp=n_comp),
                                        iter=niter, chains=4)
            fits[dobj.name].append(fit_beta)
            try:
                print(fit_beta)
            except OverflowError:
                print("Can't print! %s %d unconstrained beta" % (dobj.name, n_comp))

            fit_b2 = greybody_b2_model.sampling(data=makeStanDict(dobj, N_comp=n_comp),
                                        iter=2000, chains=4)
            fits[dobj.name].append(fit_b2)
            try:
                print(fit_b2)
            except OverflowError:
                print("Can't print! %s %d beta=2"  % (dobj.name, n_comp))



IRAS00188-0856
Inference for Stan model: anon_model_62ed89537f2de8a0302771c2118de22b.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
amplitude[0]   0.38  1.4e-4 3.4e-3   0.38   0.38   0.38   0.39   0.39  617.0   1.01
T[0]          35.05    0.04    1.1  32.97  34.29  35.03  35.78  37.29  643.0    1.0
beta[0]        1.74  2.5e-3   0.06   1.62   1.69   1.74   1.78   1.85  571.0   1.01
lp__         -10.05    0.05   1.26 -13.19 -10.62  -9.73  -9.16  -8.64  692.0    1.0

Samples were drawn using NUTS(diag_e) at Mon Aug 10 17:20:11 2015.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).
Inference for Stan model: anon_model_1dbccaaf2b16ec8ec87279bc6af8951f.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per

In [8]:
import pickle
with open("greybody_model_stan.pkl", "wb") as f:
    pickle.dump(greybody_model, f)
with open("greybody_b2_model_stan.pkl", "wb") as f:
    pickle.dump(greybody_b2_model, f)
with open("greybody_fits_stan.pkl", "wb") as f:
    pickle.dump(fits, f)



/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/pickle.py:306: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  rv = reduce(self.proto)
